# Update HBV & D-RR

The areas of HBV & D-RR are updated by this Notebook for a created scenario. The result is a adjusted PAVED.3B file for the paved areas in D-RR and a laterals.txt file, containing the unpaved areas.

## Define the name of the scenario

In [1]:
Scenario = '3e3'

## Import the required packages

In [2]:
import pandas as pd
import numpy as np

## Open the saved attribute table of the new paved areas

In [3]:
data = pd.read_csv(f'Scenarios/Scenario{Scenario}.csv',sep=',',index_col=[0])

In [4]:
data

,area,area_name
id,,
0,2698887,13.001_02
1,334381,13.001_03
2,246535,13.001_04
3,2731415,13.001_05
4,2644091,13.001_06
5,25851841,13.001_B
6,2327856,13.007
7,3580024,13.008
8,1823655,13.010


## Determine the increase of paved area per polygon

70% of the added urban areas is paved for the category 112: Discontinuous urban fabric.

In [5]:
data.area = data.area * 0.7

In [6]:
data

,area,area_name
id,,
0,1889220.9,13.001_02
1,234066.7,13.001_03
2,172574.5,13.001_04
3,1911990.5,13.001_05
4,1850863.7,13.001_06
5,18096288.7,13.001_B
6,1629499.2,13.007
7,2506016.8,13.008
8,1276558.5,13.010


## Read the adjustment ratios

The adjustment ratios are derived in A) Preparation/1) Determine Areas. The adjustment ratios show how the new paved area is divided over the D-RR nodes. Or have the adjustment of a D-RR node is divided over the subsubcatchments.

In [7]:
subcatch2drr = pd.read_csv('Adjustment Ratios/Subcatch2drr.csv',sep=';')

In [8]:
subcatch2drr

,subsubcatchment,frac,pav_code
0,10.005,0.500000,pav_1667
1,10.005,0.500000,pav_1668
2,10.012,0.654273,pav_1666
3,10.012,0.172864,pav_1667
4,10.012,0.172864,pav_1668
...,...,...,...
356,9.Q.xx,0.071679,pav_2289
357,9.Q.xx,0.071679,pav_2298
358,9.Q.xx,0.033736,pav_2287
359,9.Q.xx,0.070402,pav_2291


In [9]:
drr2subcatch = pd.read_csv('Adjustment Ratios/Drr2subcatch.csv',sep=';')

In [10]:
drr2subcatch

,pav_code,ar_paved,total_paved,fractie,subsubcatchment
0,pav_1619,607.925,75040.0,0.008101,10.001_12
1,pav_1619,17862.530,75040.0,0.238040,10.001_13
2,pav_1619,931.113,75040.0,0.012408,10.027U
3,pav_1619,20974.512,75040.0,0.279511,13.001_01
4,pav_1619,16115.663,75040.0,0.214761,13.001_02
...,...,...,...,...,...
356,pav_hombourg,39900.000,39900.0,1.000000,13.001_B
357,pav_kelmis1,258340.000,636380.0,0.405952,10.001_B
358,pav_kelmis2,258340.000,636380.0,0.405952,10.001_B
359,pav_montzen,39900.000,636380.0,0.062698,10.001_B


## Increased paved area per subsubcatchment

Determine how much paved area is added per subsubcatchment

In [11]:
cols = data.area_name.unique()

a = []
b = []

for col in cols:
    a.append(col)
    b.append(data.loc[data.area_name==col].area.sum())
    
data2 = pd.DataFrame({'subsubcatchment': a, 'paved area': b})

In [12]:
data2

,subsubcatchment,paved area
0,13.001_02,1889220.9
1,13.001_03,234066.7
2,13.001_04,172574.5
3,13.001_05,1911990.5
4,13.001_06,1850863.7
5,13.001_B,18096288.7
6,13.007,1629499.2
7,13.008,2506016.8
8,13.010,1276558.5
9,13.515,1468667.2


## Paved area increase per D-RR node

The paved area increase per subsubcatchment is translated to an increase per D-RR node. The adjustment ratios of subcatch -> D-RR is used for this.

In [13]:
c = []
d = []

for i in data2.subsubcatchment:
    
    arr = subcatch2drr.loc[subcatch2drr.subsubcatchment == i].values
    for j in arr:
        c.append(j[2])
        d.append(j[1] * data2.loc[data2.subsubcatchment == j[0]]['paved area'].values[0])
    
data3 = pd.DataFrame({'area':d},index=c)
data3 = data3.groupby(level=0)['area'].sum()
data3 = pd.DataFrame(data3)

In [14]:
data3

,area
pav_1619,9.513335e+05
pav_1620,7.707621e+05
pav_1621,1.223334e+06
pav_1623,5.273193e+05
pav_1625,7.707621e+05
pav_1633,3.819440e+05
pav_1635,3.819440e+05
pav_1638,4.635836e+05
pav_1640,9.410174e+03
pav_1641,9.410174e+03


## New total paved area per D-RR node

The new total paved area per D-RR node is estimated here.

### Load the original PAVED.3B file

This file contains the parameters for D-RR, per node.

In [15]:
nodes = pd.read_csv('paved/paved_org.3B',delimiter=' ',names=[
                'PAVE','id','id_name','ar','area','lv','street level','sd','storage identification',
                'ss','sewer system','qc','constant capacity','sewer pump capacity','DWA pump capacity',
                'qo','pump to 1','pump to 2','ms','meteostation','is','salt concentration','np','people',
                'dw','DWA identification','ro','runoff option','ru','runoff delay factor','qh','Qh-relation','pave'])
nodes.index = nodes.id_name.str.strip("'")

In [16]:
nodes

,PAVE,id,id_name,ar,area,lv,street level,sd,storage identification,ss,...,people,dw,DWA identification,ro,runoff option,ru,runoff delay factor,qh,Qh-relation,pave
id_name,,,,,,,,,,,,,,,,,,,,,
pav_1619,PAVE,id,'pav_1619',ar,75040.0,lv,92.40,sd,'sto_pav_1619',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1620,PAVE,id,'pav_1620',ar,8915.0,lv,215.82,sd,'sto_pav_1620',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.01,qh,'',pave
pav_1625,PAVE,id,'pav_1625',ar,8915.0,lv,215.82,sd,'sto_pav_1625',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1621,PAVE,id,'pav_1621',ar,16790.0,lv,139.70,sd,'sto_pav_1621',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1623,PAVE,id,'pav_1623',ar,5090.0,lv,123.18,sd,'sto_pav_1623',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pav_kelmis2,PAVE,id,'pav_kelmis2',ar,258340.0,lv,172.50,sd,'sto_pav_kelmis2',ss,...,0,dw,'Def_DWA',NaN,NaN,NaN,NaN,NaN,NaN,NaN
pav_montzen,PAVE,id,'pav_montzen',ar,39900.0,lv,161.50,sd,'sto_pav_montzen',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_gemmenich,PAVE,id,'pav_gemmenich',ar,39900.0,lv,142.00,sd,'sto_pav_gemmenich',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.01,qh,'',pave


### Create a dataframe containing the original paved areas per node

In [17]:
data4 = pd.DataFrame({'Old area':nodes.area})
data4['Increase area'] = np.zeros(len(data4))

In [18]:
data4

,Old area,Increase area
id_name,,
pav_1619,75040.0,0.0
pav_1620,8915.0,0.0
pav_1625,8915.0,0.0
pav_1621,16790.0,0.0
pav_1623,5090.0,0.0
...,...,...
pav_kelmis2,258340.0,0.0
pav_montzen,39900.0,0.0
pav_gemmenich,39900.0,0.0


### Add the new paved area to the dataframe and calculate the new paved areas per D-RR node

In [19]:
for i in data3.index:
    data4['Increase area'].loc[i] = data3.loc[i].area
    
data4['New area'] = data4['Old area'] + data4['Increase area']

In [20]:
data4

,Old area,Increase area,New area
id_name,,,
pav_1619,75040.0,9.513335e+05,1.026374e+06
pav_1620,8915.0,7.707621e+05,7.796771e+05
pav_1625,8915.0,7.707621e+05,7.796771e+05
pav_1621,16790.0,1.223334e+06,1.240124e+06
pav_1623,5090.0,5.273193e+05,5.324093e+05
...,...,...,...
pav_kelmis2,258340.0,0.000000e+00,2.583400e+05
pav_montzen,39900.0,0.000000e+00,3.990000e+04
pav_gemmenich,39900.0,0.000000e+00,3.990000e+04


### Apply the new paved areas to the PAVED.3B file

In [21]:
nodes.area = data4['New area']

In [22]:
nodes

,PAVE,id,id_name,ar,area,lv,street level,sd,storage identification,ss,...,people,dw,DWA identification,ro,runoff option,ru,runoff delay factor,qh,Qh-relation,pave
id_name,,,,,,,,,,,,,,,,,,,,,
pav_1619,PAVE,id,'pav_1619',ar,1.026374e+06,lv,92.40,sd,'sto_pav_1619',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1620,PAVE,id,'pav_1620',ar,7.796771e+05,lv,215.82,sd,'sto_pav_1620',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.01,qh,'',pave
pav_1625,PAVE,id,'pav_1625',ar,7.796771e+05,lv,215.82,sd,'sto_pav_1625',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1621,PAVE,id,'pav_1621',ar,1.240124e+06,lv,139.70,sd,'sto_pav_1621',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_1623,PAVE,id,'pav_1623',ar,5.324093e+05,lv,123.18,sd,'sto_pav_1623',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pav_kelmis2,PAVE,id,'pav_kelmis2',ar,2.583400e+05,lv,172.50,sd,'sto_pav_kelmis2',ss,...,0,dw,'Def_DWA',NaN,NaN,NaN,NaN,NaN,NaN,NaN
pav_montzen,PAVE,id,'pav_montzen',ar,3.990000e+04,lv,161.50,sd,'sto_pav_montzen',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.05,qh,'',pave
pav_gemmenich,PAVE,id,'pav_gemmenich',ar,3.990000e+04,lv,142.00,sd,'sto_pav_gemmenich',ss,...,0,dw,'Def_DWA',ro,1.0,ru,0.01,qh,'',pave


### Save the changed PAVED.3B file

In [23]:
nodes.to_csv(f'Paved/PAVED_{Scenario}.3B', header=False, index=False, sep=' ')

## New unpaved area per subsubcatchment

### Read the subsubcatchment areas

In [24]:
laterals = pd.read_csv('Laterals/laterals_org.txt',index_col=[0])

In [25]:
laterals

,id,subcatchment,area
name,,,
9.Q.xx,9.Q.xx,Meerssen,5.91
10.001_B,10.001_B,Sippenaeken,122.65
10.001_01,10.001_01,Meerssen,1.83
10.001_02,10.001_02,Meerssen,2.04
10.001_03,10.001_03,Meerssen,3.17
...,...,...,...
13.008,13.008,Gulp,3.52
13.010,13.010,Gulp,1.76
13.515,13.515,Gulp,2.10


### Calculate the new unpaved areas

The added paved area as data2 is subtracted from the unpaved areas in laterals.

In [26]:
for index, row in data2.iterrows():
    code = row['subsubcatchment']
    value = laterals.loc[code, 'area'] - row['paved area'] / 1000000
    laterals.loc[code, 'area'] = value

In [27]:
laterals

,id,subcatchment,area
name,,,
9.Q.xx,9.Q.xx,Meerssen,5.910000
10.001_B,10.001_B,Sippenaeken,122.650000
10.001_01,10.001_01,Meerssen,1.830000
10.001_02,10.001_02,Meerssen,2.040000
10.001_03,10.001_03,Meerssen,3.170000
...,...,...,...
13.008,13.008,Gulp,1.013983
13.010,13.010,Gulp,0.483442
13.515,13.515,Gulp,0.631333


### Save the new unpaved areas

In [28]:
laterals.to_csv(f'Laterals/laterals_scenario{Scenario}.txt')